Preparing to submit wold stranded samples....

This got a little messy because a couple libraries needed a top up.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path
import configparser

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp
from htsworkflow.util.api import (
    add_auth_options,
    make_auth_from_opts,
    HtswApi,
)

In [4]:
config = configparser.ConfigParser()
config.read([os.path.expanduser('~/.htsworkflow.ini'),
             '/etc/htsworkflow.ini'
             ])

SECTION = 'sequence_archive'
if config.has_section(SECTION):
    apiid = config.get(SECTION, 'apiid')
    apikey = config.get(SECTION, 'apikey')
    apihost = config.get(SECTION, 'host')

auth = {'apiid': apiid, 'apikey': apikey }
htsw = HtswApi(apihost, auth)

In [5]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/altius-biosamples.xlsx').expanduser()
engine=None
#engine='odf'

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [6]:
award = 'UM1HG009443'

# Lookup biosample ontologies

Lookup any biosample ontologies that are already present

In [9]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
        biosample.loc[i, 'source'] = obj['source']['@id']
            
biosample

KeyboardInterrupt: 

In [8]:
#biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Biosamples

In [9]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)

for i, row in biosample.iterrows():
    if not pandas.isnull(row.accession) and row.accession.startswith('E'):
        obj = server.get_json(row.accession)
        biosample_ontology = obj['biosample_ontology']
        if isinstance(biosample_ontology, dict):
            biosample.loc[i, 'biosample_ontology'] = biosample_ontology['@id']
            biosample.loc[i, 'biosample_term_name:skip'] = biosample_ontology['term_name']
        biosample.loc[i, 'source'] = obj['source']['@id']
            
biosample

,uuid,accession,aliases:array,biosample_ontology,biosample_term_name:skip,description,library_id:skip,cDNA_sample:skip,organism,source,lab,award
0,NaN,ENCBS648HXY,NaN,/biosample-types/cell_line_EFO_0010807/,F121-9,LRGASP_mouse ES-1_short_read_validation,23013,NaN,/organisms/mouse/,/sources/david-gilbert/,barbara-wold,UM1HG009443
1,NaN,ENCBS951CRC,NaN,/biosample-types/cell_line_EFO_0010807/,F121-9,LRGASP_mouse ES-2_short_read_validation,23014,NaN,/organisms/mouse/,/sources/david-gilbert/,barbara-wold,UM1HG009443
2,NaN,ENCBS418RDP,NaN,/biosample-types/cell_line_EFO_0010807/,F121-9,LRGASP_mouse ES-3_short_read_validation,23015,NaN,/organisms/mouse/,/sources/david-gilbert/,barbara-wold,UM1HG009443
3,NaN,ENCBS944CBA,NaN,/biosample-types/cell_line_EFO_0009747/,WTC11,LRGASP_WTC11-1_short_read_validation,23016,NaN,/organisms/human/,/sources/yiing-lin/,barbara-wold,UM1HG009443
4,NaN,ENCBS593PKA,NaN,/biosample-types/cell_line_EFO_0009747/,WTC11,LRGASP_WTC11-2_short_read_validation,23017,NaN,/organisms/human/,/sources/yiing-lin/,barbara-wold,UM1HG009443
5,NaN,ENCBS474NOC,NaN,/biosample-types/cell_line_EFO_0009747/,WTC11,LRGASP_WTC11-3_short_read_validation,23018,NaN,/organisms/human/,/sources/yiing-lin/,barbara-wold,UM1HG009443
6,NaN,NaN,NaN,NaN,NaN,LRGASP_H1ES-1_short_read_validation,23019,NaN,/organisms/human/,/sources/rene-maehr/,barbara-wold,UM1HG009443
7,NaN,NaN,NaN,NaN,NaN,LRGASP_H1ES-2_short_read_validation,23020,NaN,/organisms/human/,/sources/rene-maehr/,barbara-wold,UM1HG009443
8,NaN,NaN,NaN,NaN,NaN,LRGASP_H1ES-3_short_read_validation,23021,NaN,/organisms/human/,/sources/rene-maehr/,barbara-wold,UM1HG009443
9,NaN,NaN,NaN,NaN,NaN,LRGASP_H1_endo-1_short_read_validation,23022,NaN,/organisms/human/,/sources/rene-maehr/,barbara-wold,UM1HG009443


In [12]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosample', header=0, engine=engine)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True, 
                            dry_run=False, 
                            validator=validator)
print(len(created))

Reponse {'status': 'success', '@type': ['result'], '@graph': [{'accession': 'ENCBS421OHT', 'aliases': [], 'schema_version': '25', 'status': 'in progress', 'lab': '/labs/barbara-wold/', 'award': '/awards/UM1HG009443/', 'date_created': '2021-03-19T00:22:12.604719+00:00', 'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/', 'documents': [], 'references': [], 'source': '/sources/jackson-labs/', 'biosample_ontology': '/biosample-types/tissue_UBERON_0001388/', 'genetic_modifications': [], 'alternate_accessions': [], 'description': 'B6Cast F1 PND4 3x Females 06 gastrocnemius for Altius Stam core', 'treatments': [], 'dbxrefs': [], 'donor': '/mouse-donors/ENCDO509HIY/', 'organism': '/organisms/mouse/', 'internal_tags': [], 'date_obtained': '2020-01-31', 'starting_amount': 3, 'starting_amount_units': 'items', 'model_organism_age': '4', 'model_organism_age_units': 'day', '@id': '/biosamples/ENCBS421OHT/', '@type': ['Biosample', 'Item'], 'uuid': '525dab87-041d-4d64-83d1-07923ba61e3d', '

In [13]:
created

[{'accession': 'ENCBS421OHT',
  'aliases': [],
  'schema_version': '25',
  'status': 'in progress',
  'lab': '/labs/barbara-wold/',
  'award': '/awards/UM1HG009443/',
  'date_created': '2021-03-19T00:22:12.604719+00:00',
  'submitted_by': '/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/',
  'documents': [],
  'references': [],
  'source': '/sources/jackson-labs/',
  'biosample_ontology': '/biosample-types/tissue_UBERON_0001388/',
  'genetic_modifications': [],
  'alternate_accessions': [],
  'description': 'B6Cast F1 PND4 3x Females 06 gastrocnemius for Altius Stam core',
  'treatments': [],
  'dbxrefs': [],
  'donor': '/mouse-donors/ENCDO509HIY/',
  'organism': '/organisms/mouse/',
  'internal_tags': [],
  'date_obtained': '2020-01-31',
  'starting_amount': 3,
  'starting_amount_units': 'items',
  'model_organism_age': '4',
  'model_organism_age_units': 'day',
  '@id': '/biosamples/ENCBS421OHT/',
  '@type': ['Biosample', 'Item'],
  'uuid': '525dab87-041d-4d64-83d1-07923ba61e3d',
  'sex':

In [15]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

/usr/lib/python3/dist-packages/xlsxwriter/worksheet.py:358: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if token is '':
/usr/lib/python3/dist-packages/xlsxwriter/worksheet.py:2437: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if options['min_type'] is 'min' and options['min_value'] == 0:
/usr/lib/python3/dist-packages/xlsxwriter/worksheet.py:2440: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if options['max_type'] is 'max' and options['max_value'] == 0:
/usr/lib/python3/dist-packages/xlsxwriter/worksheet.py:4999: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if props[i]['type'] is 'number':
/usr/lib/python3/dist-packages/xlsxwriter/worksheet.py:6827: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if data_bar['bar_axis_position'] is not 'none':
/usr/lib/python3/dist-packages/xlsxwriter/worksheet.py:6862: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if data_bar['bar_direction'] is 'left':
/usr/lib/python3/dist-pack

# Register Libraries

In [ ]:
print(spreadsheet_name)
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Library', header=0, engine=engine)
created = server.post_sheet('/libraries/', 
                            libraries,
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

In [ ]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [ ]:
print(server.server)
experiments = pandas.read_excel(spreadsheet_name, sheet_name='Experiment', header=0, engine=engine)
experiments = experiments[experiments['accession'] != 'barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

In [ ]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [ ]:
print(server.server)
print(spreadsheet_name)
replicates = pandas.read_excel(spreadsheet_name, sheet_name='Replicate', header=0, engine=engine)
replicates = replicates[replicates['uuid'] != 'barbara approval needed']
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

In [ ]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Check Files

In [ ]:
files = pandas.read_excel(spreadsheet_name, sheet_name='File', header=0, engine=engine)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True, validator=validator)
print(len(created))